In [12]:
import requests
from requests import HTTPError,JSONDecodeError
from pprint import pprint
import csv
from datetime import datetime

def get_aqi()->list[dict]:
    try:
        url = 'https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=af57253c-e838-46da-a1f5-12b43afd75f3&limit=1000&sort=ImportDate desc&format=JSON'
        response = requests.get(url,verify=False)
        response.raise_for_status()
        data_list:list= response.json()['records'] #data_list是每一個站的的資料
        return data_list
    except HTTPError as e:
        raise Exception(f"HTTPError:{e}")
    except JSONDecodeError:
        raise Exception(f"內容不是json的格式")
    except Exception:
        raise Exception("不知的錯誤")

def filter_data(site_list:list[dict])->list[dict]:
    '''過濾資料,只要sitename,country,aqi,status'''
    filter_data:list = []
    for site in site_list:
        new_site:dict = dict()
        new_site["站點名稱"] = site['sitename']
        new_site["城市名稱"] = site['county']
        new_site['aqi'] = site['aqi']
        new_site['狀態'] = site['status']
        filter_data.append(new_site)
    return filter_data

def save_csv(data:list[dict]):
    # 產生檔案名稱(包含時間戳記)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"aqi_data_{timestamp}.csv"
    
    # 使用 csv.DictWriter 寫入資料
    if data:
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            # 取得欄位名稱
            fieldnames = data[0].keys()
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # 寫入表頭
            writer.writeheader()
            
            # 寫入所有資料列
            writer.writerows(data)
        
        print(f"資料已儲存至 {filename}")
    else:
        print("沒有資料可儲存")
    
   

def main():
    # 下載資料
    try:
        site_list = get_aqi()
    except Exception as e:
        print(e)
        return  #如果在function出錯了,直接return,'提早離開'
    else:
        print("下載成功")

    # 過濾資料
    sites = filter_data(site_list)     
    
    # 儲存成csv檔
    save_csv(sites)

if __name__ == "__main__":
    main()


/opt/conda/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


下載成功
資料已儲存至 aqi_data_20251120_122124.csv
